In [227]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [228]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [229]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [230]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [231]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [232]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [233]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [234]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [235]:
_media_file_stream = open(media_pk2_path, 'rb')

In [236]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [237]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [238]:
blowfish_checksum_decoded = "Joymax Pak File"

In [239]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [240]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [241]:
Root = read_blocks_at(256)

In [242]:
blocks_in_memory = {
  "": Root
}

In [243]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [244]:
server_dep = os.path.join('server_dep', 'silkroad', 'textdata')
server_dep

'server_dep\\silkroad\\textdata'

In [245]:

teleport_building_data = dict()

## TeleportBuilding.txt

In [246]:
teleport_building = os.path.join(server_dep, 'TeleportBuilding.txt')

In [247]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  


def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value

In [248]:
def safe_get(values, index, default="0"):
    return values[index] if index < len(values) else default

lines = get_lines(teleport_building)
for idx, line in enumerate(lines):
    values = line.split('\t')
    
    Service = convert_string(safe_get(values, 0))
    ID = convert_string(safe_get(values, 1))
    CodeName = convert_string(safe_get(values, 2))
    print(CodeName)
    ObjName = convert_string(safe_get(values, 3))
    NameStrId = convert_string(safe_get(values, 5))
    CashItem = convert_string(safe_get(values, 7))
    Bionic = convert_string(safe_get(values, 8))
    TypeId1 = convert_string(safe_get(values, 9))
    TypeId2 = convert_string(safe_get(values, 10))
    TypeId3 = convert_string(safe_get(values, 11))
    TypeId4 = convert_string(safe_get(values, 12))
    Country = convert_string(safe_get(values, 14))
    Rarity = convert_string(safe_get(values, 15))
    CanDrop = convert_string(safe_get(values, 20))
    CanUSe = convert_string(safe_get(values, 24))
    RequestLevelType1 = convert_string(safe_get(values, 32))
    RequestLevel1 = convert_string(safe_get(values, 33))
    RequestLevelType2 = convert_string(safe_get(values, 34))
    RequestLevel2 = convert_string(safe_get(values, 35))
    RequestLevelType3 = convert_string(safe_get(values, 36))
    RequestLevel3 = convert_string(safe_get(values, 37))
    RequestLevelType4 = convert_string(safe_get(values, 38))
    RequestLevel4 = convert_string(safe_get(values, 39))
    Speed1 = convert_string(safe_get(values, 46))
    Speed2 = convert_string(safe_get(values, 47))
    AssicFileIcon = convert_string(safe_get(values, 54))
    
    Level = convert_string(safe_get(values, 57))
    CharGender = convert_string(safe_get(values, 58))
    MaxHealth = convert_string(safe_get(values, 59))
    MaxMP = convert_string(safe_get(values, 60))
    InventorySize = convert_string(safe_get(values, 61))
    CanStore_TID1 = convert_string(safe_get(values, 62))
    CanStore_TID2 = convert_string(safe_get(values, 63))
    CanStore_TID3 = convert_string(safe_get(values, 64))
    CanStore_TID4 = convert_string(safe_get(values, 65))
    CanBeVehicle = convert_string(safe_get(values, 66))
    CanControl = convert_string(safe_get(values, 67))
    Param2 = convert_string(safe_get(values, 66))
    Desc2 = convert_string(safe_get(values, 67))
    DamagePortion = convert_string(safe_get(values, 68))
    MaxPassenger = convert_string(safe_get(values, 69))
    Param4 = convert_string(safe_get(values, 66))
    Desc4 = convert_string(safe_get(values, 67))
    
    IsDimensionPillar = NameStrId == "SN_MOB_GOD_PILLAR"
    IsSummonFlower = str(CodeName).startswith("STRUCTURE_SUMMON_FLOWER_")
    IsEventMob = str(CodeName).startswith("MOB_EV")
    IsPandora = TypeId2 == 2 and TypeId3 == 1 and TypeId4 == 5

    teleport_building_data[ID] = {
        "Service": Service,
        "CodeName": CodeName,
        "ObjName": ObjName,
        "NameStrId": NameStrId,
        "CashItem": CashItem,
        "Bionic": Bionic,
        "TypeId1": TypeId1,
        "TypeId2": TypeId2,
        "TypeId3": TypeId3,
        "TypeId4": TypeId4,
        "Country": Country,
        "Rarity": Rarity,
        "CanDrop": CanDrop,
        "CanUSe": CanUSe,
        "RequestLevelType1": RequestLevelType1,
        "RequestLevel1": RequestLevel1,
        "RequestLevelType2": RequestLevelType2,
        "RequestLevel2": RequestLevel2,
        "RequestLevelType3": RequestLevelType3,
        "RequestLevel3": RequestLevel3,
        "RequestLevelType4": RequestLevelType4,
        "RequestLevel4": RequestLevel4,
        "Speed1": Speed1,
        "Speed2": Speed2,
        "AssicFileIcon": AssicFileIcon,
        "Level": Level,
        "CharGender": CharGender,
        "MaxHealth": MaxHealth,
        "MaxMP": MaxMP,
        "InventorySize": InventorySize,
        "CanStore_TID1": CanStore_TID1,
        "CanStore_TID2": CanStore_TID2,
        "CanStore_TID3": CanStore_TID3,
        "CanStore_TID4": CanStore_TID4,
        "CanBeVehicle": CanBeVehicle,
        "CanControl": CanControl,
        "Param2": Param2,
        "Desc2": Desc2,
        "DamagePortion": DamagePortion,
        "MaxPassenger": MaxPassenger,
        "Param4": Param4,
        "Desc4": Desc4,
        "IsDimensionPillar": IsDimensionPillar,
        "IsSummonFlower": IsSummonFlower,
        "IsEventMob": IsEventMob,
        "IsPandora": IsPandora
    }


STORE_EVENTZONE_DEFAULT
STORE_CH_GATE
STORE_WC_GATE
STORE_KT_GATE
STORE_TD_GATE
STORE_EU_GATE
STORE_CA_GATE
INS_QUEST_TELEPORT
STORE_TQ_SNAKEGATE
STORE_TQ_6FGATE2
STORE_TQ_6FGATE3
STORE_TQ_6FGATE4
STORE_TQ_6FGATE1
STORE_CH_FORT_GATE1
STORE_CH_FORT_GATE2
STORE_CH_FORT_GATE3
STORE_JA_FORT_GATE1
STORE_JA_FORT_GATE2
STORE_JA_FORT_GATE3
STORE_JA_CHARGE_GATE
STORE_JA_GLORY_GATE
STORE_JA_REVIVAL_GATE
STORE_BD_FORT_GATE1
STORE_BD_FORT_GATE2
STORE_BJ_FORT_GATE1
STORE_BJ_FORT_GATE2
STORE_BJ_FORT_GATE3
STORE_BJ_CHARGE_GATE
STORE_BJ_GLORY_GATE
STORE_BJ_REVIVAL_GATE
STORE_KT_FORT_GATE1
STORE_KT_FORT_GATE2
STORE_KT_FORT_GATE3
STORE_HT_FORT_GATE1
STORE_HT_FORT_GATE2
STORE_HT_FORT_GATE3
STORE_HT_FORT_GATE4
STORE_HT_CHARGE_GATE
STORE_HT_GLORY_GATE
STORE_HT_REVIVAL_GATE
STORE_RC_TAHOMET_GATE
STORE_ROC_FRONT_GATE
STORE_TOMB_GATE_IN
STORE_TOMB_GATE_OUT_1
STORE_TOMB_GATE_OUT_2
STORE_TOMB_GATE_OUT_3
STORE_TOMBOSIRIS_GATE_OUT_1
STORE_TOMBOSIRIS_GATE_OUT_2
STORE_TOMBOSIRIS_GATE_OUT_3
STORE_DESERT_MERCHANT_HUN

In [249]:
teleport_building_data

{33714: {'Service': 1,
  'CodeName': 'STORE_EVENTZONE_DEFAULT',
  'ObjName': '???? ? ?? ????',
  'NameStrId': 'xxx',
  'CashItem': 0,
  'Bionic': 0,
  'TypeId1': 4,
  'TypeId2': 2,
  'TypeId3': 1,
  'TypeId4': 1,
  'Country': 3,
  'Rarity': 0,
  'CanDrop': 0,
  'CanUSe': 0,
  'RequestLevelType1': -1,
  'RequestLevel1': 0,
  'RequestLevelType2': -1,
  'RequestLevel2': 0,
  'RequestLevelType3': -1,
  'RequestLevel3': 0,
  'RequestLevelType4': -1,
  'RequestLevel4': 0,
  'Speed1': 0,
  'Speed2': 0,
  'AssicFileIcon': 'xxx',
  'Level': 0,
  'CharGender': 0,
  'MaxHealth': 0,
  'MaxMP': 0,
  'InventorySize': 0,
  'CanStore_TID1': 0,
  'CanStore_TID2': 0,
  'CanStore_TID3': 0,
  'CanStore_TID4': 0,
  'CanBeVehicle': 0,
  'CanControl': 0,
  'Param2': 0,
  'Desc2': 0,
  'DamagePortion': 0,
  'MaxPassenger': 0,
  'Param4': 0,
  'Desc4': 0,
  'IsDimensionPillar': False,
  'IsSummonFlower': False,
  'IsEventMob': False,
  'IsPandora': False},
 2094: {'Service': 1,
  'CodeName': 'STORE_CH_GATE',
 

## TeleportData.txt

In [250]:
teleport_data = []

In [251]:
teleport_data_file = os.path.join(server_dep, 'TeleportData.txt')
teleport_data_file

'server_dep\\silkroad\\textdata\\TeleportData.txt'

In [252]:
lines = get_lines(teleport_data_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')

      Service = values[0]
      Id = values[1]
      CodeGame = values[2]  
      AssocRefObjId = values[3]
      ZoneName128 = values[4]
      GenRegionID = values[5]
      GenPos_X = values[6]
      GenPos_Z = values[7]
      GenPos_Y = values[8]
      GenAreaRadius = values[9]
      CanBeResurrectPos = values[10]
      CanGotoResurrectPos = values[11]

      obj = {
        "Service": Service,
        "Id": Id,
        "CodeGame": CodeGame,
        "AssocRefObjId": AssocRefObjId,
        "ZoneName128": ZoneName128,
        "GenRegionID": GenRegionID,
        "GenPos_X": GenPos_X,
        "GenPos_Z": GenPos_Z,
        "GenPos_Y": GenPos_Y,
        "GenAreaRadius": GenAreaRadius,
        "CanBeResurrectPos": CanBeResurrectPos,
        "CanGotoResurrectPos": CanGotoResurrectPos
      }

      teleport_data.append(obj)
    except Exception as e:
      continue

In [253]:
teleport_data

[{'Service': '1',
  'Id': '1',
  'CodeGame': 'GATE_CH',
  'AssocRefObjId': '2094',
  'ZoneName128': 'SN_ZONE_22001',
  'GenRegionID': '25000',
  'GenPos_X': '969',
  'GenPos_Z': '0',
  'GenPos_Y': '1369',
  'GenAreaRadius': '150',
  'CanBeResurrectPos': '1',
  'CanGotoResurrectPos': '0'},
 {'Service': '1',
  'Id': '2',
  'CodeGame': 'GATE_WC',
  'AssocRefObjId': '2095',
  'ZoneName128': 'SN_ZONE_22002',
  'GenRegionID': '26265',
  'GenPos_X': '957',
  'GenPos_Z': '-80',
  'GenPos_Y': '1508',
  'GenAreaRadius': '200',
  'CanBeResurrectPos': '1',
  'CanGotoResurrectPos': '0'},
 {'Service': '1',
  'Id': '3',
  'CodeGame': 'GATE_NPC_CH_FERRY',
  'AssocRefObjId': '2011',
  'ZoneName128': 'SN_ZONE_21002',
  'GenRegionID': '24993',
  'GenPos_X': '560',
  'GenPos_Z': '140',
  'GenPos_Y': '1460',
  'GenAreaRadius': '200',
  'CanBeResurrectPos': '0',
  'CanGotoResurrectPos': '0'},
 {'Service': '1',
  'Id': '4',
  'CodeGame': 'GATE_NPC_WC_FERRY',
  'AssocRefObjId': '2056',
  'ZoneName128': 'SN_ZO

## TeleportLink.txt

In [254]:
teleport_link = []

In [255]:
teleport_link_file = os.path.join(server_dep, 'TeleportLink.txt')
teleport_link_file

'server_dep\\silkroad\\textdata\\TeleportLink.txt'

In [256]:
lines = get_lines(teleport_link_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')

      Service = values[0]
      OwnerTeleport = values[1]
      TargetTeleport = values[2]  
      Fee = values[3]
      RestrictBindMethod = values[4]
      CheckResult = values[5]
      Restrict1 = values[6]
      Data1_1 = values[7]
      Data1_2 = values[8]
      Restrict2 = values[9]
      Data2_1 = values[10]
      Data2_2 = values[11]
      Restrict3 = values[12]
      Data3_1 = values[13]
      Data3_2 = values[14]
      Restrict4 = values[15]
      Data4_1 = values[16]
      Data4_2 = values[17]
      Restrict5 = values[18]
      Data5_1 = values[19]
      Data5_2 = values[20]

      obj = {
        "Service": Service,
        "OwnerTeleport": OwnerTeleport,
        "TargetTeleport": TargetTeleport,
        "Fee": Fee,
        "RestrictBindMethod": RestrictBindMethod,
        "CheckResult": CheckResult,
        "Restrict1": Restrict1,
        "Data1_1": Data1_1,
        "Data1_2": Data1_2,
        "Restrict2": Restrict2,
        "Data2_1": Data2_1,
        "Data2_2": Data2_2,
        "Restrict3": Restrict3,
        "Data3_1": Data3_1,
        "Data3_2": Data3_2,
        "Restrict4": Restrict4,
        "Data4_1": Data4_1,
        "Data4_2": Data4_2,
        "Restrict5": Restrict5,
        "Data5_1": Data5_1,
        "Data5_2": Data5_2
      }

      teleport_link.append(obj)
    except Exception as e:
      continue

In [257]:
teleport_link

[{'Service': '1',
  'OwnerTeleport': '1',
  'TargetTeleport': '2',
  'Fee': '5000',
  'RestrictBindMethod': '0',
  'CheckResult': '0',
  'Restrict1': '2',
  'Data1_1': '0',
  'Data1_2': '0',
  'Restrict2': '0',
  'Data2_1': '0',
  'Data2_2': '0',
  'Restrict3': '0',
  'Data3_1': '0',
  'Data3_2': '0',
  'Restrict4': '0',
  'Data4_1': '0',
  'Data4_2': '0',
  'Restrict5': '0',
  'Data5_1': '0',
  'Data5_2': '0'},
 {'Service': '1',
  'OwnerTeleport': '42',
  'TargetTeleport': '43',
  'Fee': '0',
  'RestrictBindMethod': '0',
  'CheckResult': '0',
  'Restrict1': '1',
  'Data1_1': '0',
  'Data1_2': '999',
  'Restrict2': '2',
  'Data2_1': '0',
  'Data2_2': '0',
  'Restrict3': '0',
  'Data3_1': '0',
  'Data3_2': '0',
  'Restrict4': '0',
  'Data4_1': '0',
  'Data4_2': '0',
  'Restrict5': '0',
  'Data5_1': '0',
  'Data5_2': '0'},
 {'Service': '1',
  'OwnerTeleport': '42',
  'TargetTeleport': '44',
  'Fee': '0',
  'RestrictBindMethod': '0',
  'CheckResult': '0',
  'Restrict1': '1',
  'Data1_1': 

## refoptionalteleport.txt

In [258]:
optional_teleports = dict()

In [259]:
ref_optional_teleport_file = os.path.join(server_dep, 'refoptionalteleport.txt')
ref_optional_teleport_file

'server_dep\\silkroad\\textdata\\refoptionalteleport.txt'

In [260]:
lines = get_lines(ref_optional_teleport_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')

      Service = values[0]
      ID = values[1]
      Region = values[4]  

      optional_teleports[ID] = {
        "Service": Service,
        "Region": Region,
        "ID": ID
      }
    except Exception as e:
      continue

In [261]:
optional_teleports

{'1': {'Service': '1', 'Region': '26959', 'ID': '1'},
 '2': {'Service': '1', 'Region': '27207', 'ID': '2'},
 '3': {'Service': '1', 'Region': '25416', 'ID': '3'},
 '4': {'Service': '1', 'Region': '26205', 'ID': '4'},
 '5': {'Service': '1', 'Region': '25957', 'ID': '5'},
 '6': {'Service': '1', 'Region': '27244', 'ID': '6'},
 '7': {'Service': '1', 'Region': '26219', 'ID': '7'},
 '8': {'Service': '1', 'Region': '26737', 'ID': '8'},
 '9': {'Service': '1', 'Region': '24431', 'ID': '9'},
 '10': {'Service': '1', 'Region': '23664', 'ID': '10'},
 '11': {'Service': '1', 'Region': '23914', 'ID': '11'},
 '12': {'Service': '1', 'Region': '22125', 'ID': '12'},
 '13': {'Service': '1', 'Region': '23156', 'ID': '13'},
 '14': {'Service': '1', 'Region': '26752', 'ID': '14'},
 '15': {'Service': '1', 'Region': '25731', 'ID': '15'},
 '16': {'Service': '1', 'Region': '26503', 'ID': '16'},
 '17': {'Service': '1', 'Region': '23687', 'ID': '17'},
 '18': {'Service': '1', 'Region': '24713', 'ID': '18'},
 '19': {'S